# example data science pipeline

let's run through a couple steps to fit several models to the adult income dataset

## imports and constants

In [ ]:
# this line configures matplotlib (the backbone of the pandas
# plotting functions) to render the graphs it makes in the 
# notebook
%matplotlib inline

# for these "import ... as ..", the alias terms (phrases after "as")
# are simply conventions. You will usually see stack overflow code
# referencing these aliases
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline
import seaborn as sns
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.externals.joblib
import sklearn.feature_selection
import sklearn.linear_model
import sklearn.model_selection
import sklearn.neural_network
import sklearn.pipeline
import sklearn.preprocessing

# this step invokes seaborn one time to make the default plot
# configurations of matplotlib less heinous
sns.set()

# this command informs the plotly module that you are connected
# to the internet but wish to run in "offline" mode (that is,
# graph things like a normal plotting library instead of sending
# everything off to plotly HQ)
plotly.offline.init_notebook_mode(connected=True)

## loading data

let's go download a relatively large dataset that is available as part of the [UCI machine learning repository](http://archive.ics.uci.edu/ml/index.php). I've chosen the [Adult](http://archive.ics.uci.edu/ml/datasets/Adult) dataset 

### keeping things simple

we *could* use the requests library to download and parse the column names (available [here](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names)), but instead I've just hard-coded them below.

also, we *could* use the pre-segregated train and test data sets as our train and test, but that would involve some data munging and cleaning that is a bit of a mess, and also results in enough data points in our final plots that we'd have to change some annoying configuration variables. instead, let's pull only the smaller training dataset, and use the `scikit-learn` train / test split function to create a test dataset of our own.

In [ ]:
columns = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'target'
]

df = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
    names=columns,
    delimiter=', ',
    index_col=False,
    engine='python'
)

let's print the first 5 records of that dataset `df`

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

ugh. those column names... those dashes... abominations.

let's clean that up. I can clean up one using the following

In [ ]:
col = 'capital-gain'
col.replace('-', '_')

let's use a list comprehension to clean up the elements of `df.columns`

In [ ]:
df.columns

In [ ]:
#df.columns = [
# --------------- #
# FILL ME IN !!!! #
# --------------- #
#]

and now, the first 5 records again, to see that the column names have changed:

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

and it'd be good to know the shape of this dataframes:

In [ ]:
df.shape

as we go along, to verify that everything has worked as expected, we will use `assert` statements. `assert` is a `python` statement which will "do nothing" if the thing you are `assert`ing is `True`, and will raise an `AssertionError` if it is not. this is a way of silently verifying that code you've writen is behaving as expected. try the following, for example:

In [ ]:
assert 1 == 1

In [ ]:
assert 1 == 2

let's verify that the dataframe we've constructed so far is the right shape before moving on:

In [ ]:
assert df.shape == (32561, 15)

for the rest of this notebook, **if you fail an `assert` statement, something has gone wrong**

## pre-processing

let's put together a sequence of pre-processing operations we wish to do to our test and train datatset. Since we're going to do it to both test and train, it'd be good to build up a function along the way (so we don't have to repeat every command twice).

### dropping some columns

let's skip some of the hard work and just know, ahead of time, that the column `fnlwgt` and `education_num` should be dropped. Why?

1. `fnlwgt` is a weighting for demographic sampling, and is an estimate of how many people fall into the given category. we're not going to use this weighting, so let's get rid of it.
2. `education_num` is a numerical representation of the values in the `education` column. You could argue that you should keep this numeric column and drop the `education` column, or convert `education` into a dummy column and drop `education_num`. we'll do the latter.

In [ ]:
df = df.drop(['fnlwgt', 'education_num'], axis=1)

In [ ]:
assert df.shape[1] == 13

### converting categorical factors to numerical dummy factors

let's convert the categorical columns in this dataset to dummy variables, and then extract the numerical values into an `X` and `Y` dataset

first, let's look at the number of values for each category:

In [ ]:
for col in df.columns:
    print('{}: {}'.format(col, df[col].nunique()))

investigation of the number of valid elements (as well as a measure of common sense) leads to the following list of categorical values which should be converted into dummy variables:

In [ ]:
dummycols = [
    'workclass',
    'education',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native_country'
]

the `target` column, though categorical, is left out because we're about to use the above list to convert our categorical *predictors* into dummy variables. `pandas` provides a simple function for doing just that:

[`pd.get_dummies`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html). look at the documentation quick to figure out how it will work:

In [ ]:
pd.get_dummies?

use this function to add dummy features. in particular, set the following parameters:

1. `data`: this is the base dataframe from which we want to build *some* dummy variables
1. `dummy_na`: actually, *none* of our categories have `nan` values. you should verify this, but you can take my word for it and *not* create a dummy column for `nan` values
1. `columns`: use the list of dummy columns we generated just above

In [ ]:
df = pd.get_dummies(
    # --------------- #
    # FILL ME IN !!!! #
    # --------------- #
)
df.head()

In [ ]:
assert df.shape[1] == 107

### fixing column names again

some of those dummy variable names are pretty gnarly (dashes and capital letters abound!), so let's fix the column names one more time. we could fix a single column name via:

In [ ]:
col = 'native_country_Puerto-Rico'
col.lower().replace('-', '_')

In [ ]:
#df.columns = [
# --------------- #
# FILL ME IN !!!! #
# --------------- #
#]

In [ ]:
df.head()

In [ ]:
assert {
    _ for _ in df.columns if _.startswith('sex')
} == {'sex_female', 'sex_male'}

### fixing the target categorical variable

currently, our target looks like

In [ ]:
df.target.value_counts()

let's replace those string values with numerical targets.

there are two ways we could do this. first, we could convert this factor into a *category*, and then use the integer values (called *codes*) that are assigned to the two types:

In [ ]:
df.target.astype('category').cat.codes.head(10)

the second way we could do this is with a simple boolean expression, converted to an integer:

In [ ]:
(df.target == '>50K').head(10)

In [ ]:
(df.target == '>50K').astype(int).head(10)

either would work. I'll go with the `bool` version for this step

in `pandas`, whenever you *create* or *re-assign* features, you should use the `.loc` or `.iloc` indexers (not the `.` or `[...]` notations, as might feel more natural):

```python
# one new or re-assigned feature
df.loc[:, 'colname'] = newvalues

# several new or re-assigned features
df.loc[:, ['col1', 'col2', ...]] = newvalues
```

in the above, the items inside the `[...]` are the row index slice statement (a "`:`" character means "all rows"), and the item after the comma is the column index slice statement (a column name as a string, or a list of multiple columns).

the item on the left will have a certain number of rows and a certain number of columns; the right hand side must be of the same size.

we'll use the `loc` indexer here (we are *replacing* `target` values)

In [ ]:
df.loc[:, 'target'] = (df.target == '>50K').astype(int)

In [ ]:
assert df.target.unique().tolist() == [0, 1]

### dropping non-numeric features

let's get rid of the text features (most of which we just converted into dummy variables). this is straight-forward using the *hidden* dataframe class member function (items with one leading `_` are called "hidden" member functions) `_get_numeric_data`, which restricts a dataframe to only those columns with a *numeric* `dtype` (a `numpy` concept representing the type of data stored in a column).

In [ ]:
df = df._get_numeric_data()
df.head()

In [ ]:
assert df.shape[1] == 107

### standardizing

#### `log`-transform of monetary features

we have some monetary information, so it'd be good to `log` normalize those. Let's also normalize all numerical features!

In [ ]:
moneycols = [
    'capital_gain',
    'capital_loss'
]

to show these distributions have long tails and problematic distributions:

In [ ]:
df[moneycols].hist(figsize=(14, 6), bins=30);

we can easily calculate the `log(1 + x)` value for those columns:

In [ ]:
log1p = np.log1p(df[moneycols])
log1p.head()

what about the distributions under this transform?

In [ ]:
log1p.hist(figsize=(14, 6), bins=30);

let's use the `loc` indexer again, this time to *replace* the values of our `moneycols` with these `log`-transformed features.

in this case, we want to replace several columns, so we will write

```python
df.loc[:, listOfColumnNames] = newValuesForThoseColumns
```

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

In [ ]:
assert df.capital_gain.max() < 20

#### standardizing numerical features

let's go ahead and do the simplest thing -- standardizing *every* non-target variable. this is a little meaningless for dummy variables, but hey -- it's just an excercise, right?

In [ ]:
nottarget = [col for col in df.columns if col != 'target']

we can scale items in a dataframe using the `sklearn.preprocessing.scale` function (note: this returns a `numpy` array without column names)

In [ ]:
sklearn.preprocessing.scale(df[nottarget])

use the column list `nottarget` we just created, the `.loc` dataframe member function, and the `sklearn.preprocessing.scale` function to replace all non-target numeric values with their scaled values

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

In [ ]:
df.head()

In [ ]:
assert -2 < df.age.min() < 0

In [ ]:
assert 0 < df.age.max() < 4

## train and test split

in order to have some completely unseen data for final model validation, it's a good idea to hold out about 20 percent of your data. We can use the `sklearn.model_selection.train_test_split` function to split a single dataframe of labeled cases into two sets (train and test) of predictors and targets (`X` and `Y`), while keeping the prevalance of our target variable fixed between our training and testing `Y`.

In [ ]:
sklearn.model_selection.train_test_split?

In [ ]:
dftrain, dftest = sklearn.model_selection.train_test_split(
    df,
    test_size=0.2,
    random_state=1337,
    stratify=df.target
)

In [ ]:
df.shape

In [ ]:
dftrain.shape

In [ ]:
dftest.shape

In [ ]:
assert dftrain.shape[0] == 26048

check the prevalence of the target (`target`) in each dataset is approximately equivalent

*hint: there are several ways you could calculate this, but you should look to the `value_counts` function*

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

In [ ]:
assert 0.74 <= (dftest.target == 0).mean() <= 0.76

### `x` and `y`, at last

let's split our dataframes into `x` and `y` values for easier use in the `sklearn` world

In [ ]:
xtrain = dftrain[nottarget].values
ytrain = dftrain['target'].values
xtest = dftest[nottarget].values
ytest = dftest['target'].values

In [ ]:
assert xtrain.shape == (26048, 106)

In [ ]:
assert ytrain.shape == (26048,)

## fitting a data science pipeline

the hard part is over! let's use `scikit-learn` to create a pipeline that combines feature selection and a modelling approach, and then evaluate the results of our model against the test dataset

### feature selection

let's try recursive feature elimination with random forests

In [ ]:
# RFE with random forests
rf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=1337
)
rfe = sklearn.feature_selection.RFE(
    estimator=rf
)

### modeler

let's try a random forest, a logistic regression, and a neural net

In [ ]:
mrf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=1337,
)

### pipelines

we could call the above items sequentially:

```python
xfs = rfe.fit_transform(xtrain, ytrain)
mrf.fit(xtrain, ytrain)
```

and that'd be fine. however, `scikit-learn` has a concept of *pipelines*, sequences of events that all expose these `fit`, `transform`, and `fit_transform` methods. it allows you to chain these items together in a sort of linux-esque sequence of piped commands (hence, pipeline).

let's create a pipeline.

In [ ]:
pipeline = sklearn.pipeline.Pipeline(
    steps=[
        # a sequence of name, transformer objects
        ('rfe', rfe),
        ('random_forest', mrf)
    ]
)

fitting our model then is a simple call of the pipeline's `fit` method:

In [ ]:
pipeline.fit(xtrain, ytrain)

## plotting results

### feature importance

our pipeline has a feature selection element we can access via the `named_steps` member variable.

that fitted feature selection object has a few variables of interest:

1. `support`, accessed via `get_support()`: a boolean indicating whether or not a feature should be kep
2. `importance`: the average feature importance of a given feature across all the trees in our random forest 

In [ ]:
fs = pipeline.named_steps['rfe']

In [ ]:
fs.get_support()

In [ ]:
fs.estimator_.feature_importances_

In [ ]:
dfsupport = pd.DataFrame({
    'feature': nottarget,
    'support': fs.get_support()
})
dfsupport.head()

In [ ]:
# we only have feature importance for records where `support` is true
dfsupport.loc[
    dfsupport.support, 'importance'
] = fs.estimator_.feature_importances_
dfsupport.head()

let's look at the five most important features:

In [ ]:
dfsupport = dfsupport.sort_values(by='importance', ascending=False)
dfsupport.head()

now, let's use `plotly`'s `Bar` element to create a **horizontal** bar chart. I'll fill in the majority of the items here, but you will need to figure out how to construct the actual bar chart data element.

this might be helpful: https://plot.ly/python/horizontal-bar-charts/

In [ ]:
# drop the features which weren't chosen, and invert the sort
# order (plotly adds bars in this "top to bottom" way for
# horizontal bar charts)
nonzero = dfsupport[dfsupport.support].sort_values(by='importance')

In [ ]:
# use some features of nonzero (created above) 
# to fill in the elements of your bar chart.
# also, make sure the bar chart is horizontal!
barchart = go.Bar(
    # --------------- #
    # FILL ME IN !!!! #
    # --------------- #
)

In [ ]:
assert barchart.orientation == 'h'

In [ ]:
data = [barchart]

layout = go.Layout(
    # tall enough to include every feature we selected
    height=1200,
    # enough space to display the full feature name
    margin=go.layout.Margin(l=250),
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

### prediction results

we can also easily make predictions with the fit pipeline object -- this will take raw input data, apply feature selection, and score the records:

In [ ]:
ypred = pipeline.predict_proba(xtest)
ypred

the two columns in `ypred` are the predicted probability of the classes 0 and 1 for the target:

In [ ]:
pipeline.classes_

thus the probability of having a target value of 1 (equivalently: having a salary over $50K) is the second column

let's combine the predicted probabilities for our test set with the known labeled ground truth:

In [ ]:
dfpred = pd.DataFrame({
    'ytest': ytest,
    'ypred': ypred[:, 1]
})
dfpred.head()

let's use plotly to plot the cumulative captured response on the held out test data from the original dataframe. to do this, we will need to pick out the now-trained pipeline corresponding to our best run:

In [ ]:
dfpred = dfpred.sort_values(by='ypred', ascending=False)
ntargets = dfpred.ytest.sum()
dfpred.loc[:, 'pct_captured'] = dfpred.ytest.cumsum() / ntargets

xarr = np.array(range(dfpred.shape[0]))
yperf = np.ones(xarr.shape)
yperf[:ntargets] = np.linspace(0, 1, ntargets)

In [ ]:
data = [
    # our capture rate
    go.Scatter(
        x=xarr,
        y=dfpred.pct_captured,
        mode='lines',
        line={'width': 2},
        name='our prediction'
    ),
    # random choice
    go.Scatter(
        x=xarr,
        y=xarr / xarr.max(),
        mode='lines',
        line={
            'dash': 'dash',
            'color': 'black',
            'width': 1,
        },
        name='random'
    ),
    # perfect
    go.Scatter(
        x=xarr,
        y=yperf,
        mode='lines',
        line={
            'dash': 'dot',
            'color': 'black',
            'width': 1,
        },
        name='perfect'
    )
]

In [ ]:
# create a layout with axes labels and title
layout = go.Layout(
    title='cumulative captured response',
    xaxis={'title': 'number of records recommend and investigated'},
    yaxis={'title': 'fraction of all true cases obtained'}
)

now, create the `plotly go.Figure` object using the `data` and `layout` elements above, and create the `offline plot` (note: we did exactly this for the horizontal bar plot above)

In [ ]:
# --------------- #
# FILL ME IN !!!! #
# --------------- #

that is *pretty good*.

maybe *too pretty good*...

# you're done!

You can feel free to submit the homework if you've gotten to this point. However, what follows is a bit of an advanced digression into training multiple models, selecting the best one based on cross-validation, and evaluating that (hopefully better) model on the test data set

# **advanced**: cross-validation of multiple pipelines for model selection

we chose an arbitrary feature selection method and modelling approach above and had pretty good results from it -- lucky!

in practice, it would be better to try several different feature selection and modelling methods, and to try and find some test-agnostic way of selecting the best among them. a common approach is to create many different pipelines and for each pipeline evaluate a given metric under cross validation. The model with the best cross-validated metric score is then selected, and the final evaluation is then performed against the held out (test) data

### cross validation

we will need to do some cross validation for

1. grid searches for parameters, and
2. feature or model selection by metric scores on cross-validated samples

to accomplish this, we'll use the `sklearn.model_selection.StratifiedShuffleSplit`, an implementation of a fixed bootstrap cross validation selector.

In [ ]:
cv = sklearn.model_selection.StratifiedShuffleSplit(
    n_splits=10,
    test_size=0.2,
    random_state=1337
)

### feature selectors

let's try recursive feature selection with random forests and lasso (done using the logistic regression cross validation model with an L1 penalty

In [ ]:
# RFE with random forests
rf = sklearn.ensemble.RandomForestClassifier(n_jobs=-1, random_state=1337)
rfe = sklearn.feature_selection.RFE(
    estimator=rf
)

In [ ]:
# lasso
lr = sklearn.linear_model.LogisticRegression(
    C=.1,
    penalty='l1',
    solver='saga',
    n_jobs=-1,
    random_state=1337,
    max_iter=250
)
lasso = sklearn.feature_selection.SelectFromModel(estimator=lr)

### modelers

let's try a random forest, a logistic regression, and a neural net

In [ ]:
mrf = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=1337,
)

In [ ]:
mLogRegCv = sklearn.linear_model.LogisticRegressionCV(
    Cs=np.logspace(-3, 3, 7),
    cv=cv,
    scoring='neg_log_loss',
    n_jobs=-1,
    max_iter=500,
    random_state=1337,
    verbose=1
)

In [ ]:
mMlp = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(25,10),
    activation='logistic',
    max_iter=500,
    random_state=1337,
)

### pipelines

for each combo of feature selection method and model, let's create a pipeline. these pipelines can be passed to a model selection method to help us choose the best model among all combinations, just as we did above.

we don't *need* names for each step (they will be created as the class names of the objects passed in), but it's nice. we can use the `zip` function to combine lists of names and lists of objects. this means we can create our entire collection of pipelines using a list comprehension and some iterator functions.

given that we will have two feature selection methods and three models, that's a combination of 6 pipelines. It'd be nice to not have to perform feature selection 3 times when one is sufficient -- we can accomplish this using a cached memory feature built in to `scikit-learn`

In [ ]:
import tempfile

cachedir = tempfile.mkdtemp()
memory = sklearn.externals.joblib.Memory(location=cachedir)
print('cachedir was {}'.format(cachedir))

In [ ]:
pipelines = [
    sklearn.pipeline.Pipeline(
        steps=[
            (fsname, fs),
            (modelname, model)
        ],
        memory=memory
    )
    for (fsname, fs) in [
        ('lasso', lasso),
        ('rfe', rfe)
    ]
    for (modelname, model) in [
        ('random_forest', mrf),
        ('logistic', mLogRegCv), 
        ('neural_net', mMlp)
    ]
]

## selecting model via cross validation

I will collect the scores from each cross validation loop into a dataframe. we can then later group that data frame by model and feature selection type (columns `m` and `fs` below, resp.) to get average and standard deviation values.

In [ ]:
dfscores = pd.DataFrame()

for p in pipelines:
    fsname = p.steps[0][0]
    mname = p.steps[1][0]
    print('{} - {}'.format(fsname, mname))
    score = sklearn.model_selection.cross_validate(
        estimator=p,
        X=xtrain,
        y=ytrain,
        scoring=('accuracy', 'neg_log_loss'),
        cv=cv,
        n_jobs=-1
    )
    score['fs'] = fsname
    score['m'] = mname
    dfscoresnow = pd.DataFrame(score)
    
    dfscores = dfscores.append(dfscoresnow, ignore_index=True)
    
dfscores.head()

In [ ]:
dfscores.groupby(['fs', 'm']).mean()

In [ ]:
dfscores.groupby(['fs', 'm']).std()

suppose we want to choose the item with the best (here: largest) negative log loss on test:

In [ ]:
fs, m = dfscores.groupby(['fs', 'm']).mean().test_neg_log_loss.idxmax()

print(fs)
print(m)

## plotting results

let's use plotly to plot the cumulative captured response on the held out test data from the original dataframe. 

to do this, we will need to pick out the now-trained pipeline corresponding to our best run:

In [ ]:
p = [p for p in pipelines if fs in p.named_steps and m in p.named_steps][0]

the pipeline has *not* been fit on the full training data yet, just bootstrapped sub-samples of the training set. let's train it on the full model, and use that trained model to predict on our test dataset

In [ ]:
p.fit(xtrain, ytrain)
ypred = p.predict_proba(xtest)

In [ ]:
ypred

In [ ]:
dfpred = pd.DataFrame({
    'ytest': ytest,
    'ypred': ypred[:, 1]
})

dfpred.head()

In [ ]:
dfpred = dfpred.sort_values(by='ypred')

In [ ]:
dfpred.head()

In [ ]:
dfpred = dfpred.sort_values(by='ypred', ascending=False)
ntargets = dfpred.ytest.sum()
dfpred.loc[:, 'pct_captured'] = dfpred.ytest.cumsum() / ntargets

xarr = np.array(range(dfpred.shape[0]))
yperf = np.ones(xarr.shape)
yperf[:ntargets] = np.linspace(0, 1, ntargets)

In [ ]:
data = [
    # our capture rate
    go.Scatter(
        x=xarr,
        y=dfpred.pct_captured,
        mode='lines',
        line={'width': 2},
        name='our prediction'
    ),
    # random choice
    go.Scatter(
        x=xarr,
        y=xarr / xarr.max(),
        mode='lines',
        line={
            'dash': 'dash',
            'color': 'black',
            'width': 1,
        },
        name='random'
    ),
    # perfect
    go.Scatter(
        x=xarr,
        y=yperf,
        mode='lines',
        line={
            'dash': 'dot',
            'color': 'black',
            'width': 1,
        },
        name='perfect'
    )
]

In [ ]:
# create a layout with axes labels and title
layout = go.Layout(
    title='cumulative captured response',
    xaxis={'title': 'number of records recommend and investigated'},
    yaxis={'title': 'fraction of all true cases obtained'}
)

In [ ]:
# create a figure to join the above
fig = go.Figure(
    data=data,
    layout=layout
)

In [ ]:
plotly.offline.iplot(fig)

clean up after yourself

In [ ]:
import shutil
shutil.rmtree(cachedir)